In [1]:
## Imports
import os
import numpy as np                    
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
import seaborn as sns
from skimage import io as sio
from Lab_Analyses.Spine_Analysis_v2 import spine_utilities as s_utils
from Lab_Analyses.Utilities.save_load_pickle import load_pickle
from Lab_Analyses.Video_Processing.smooth_trim_video import smooth_trim_video

sns.set()
sns.set_style("ticks")

In [2]:
# Load in output from GUI
mouse_id = "JW306"
file_name = r"G:\Analyzed_data\individual\JW306\imaging\FOV2\Early\JW306_240326_GreenCh_imaging_data.pickle"
data = load_pickle([file_name])[0]

In [3]:
# Get activity related data
spine_activity = data.activity_trace["Spine"]
spine_dFoF = data.processed_dFoF["Spine"]
#dendrite_activity = data.dendrite_calcium_activity
#dendrite_dFoF = data.dendrite_calcium_processed_dFoF
#spine_calcium = data.spine_calcium_processed_dFoF

In [4]:
# Mask active and inactive portions
def get_active_inactive(dFoF, active):
    inactive_mask = (active == 1)
    active_mask = (active == 0)
    active_dFoF = np.copy(dFoF)
    inactive_dFoF = np.copy(dFoF)
    active_dFoF[active_mask] = np.nan
    #inactive_dFoF[inactive_mask] = np.nan
    
    return active_dFoF, inactive_dFoF

In [5]:
spine_active = np.zeros(spine_activity.shape) * np.nan
spine_inactive = np.zeros(spine_activity.shape) * np.nan
#dend_active = np.zeros(dendrite_activity.shape) * np.nan
#dend_inactive = np.zeros(dendrite_activity.shape) * np.nan

for i in range(spine_activity.shape[1]):
    s_active, s_inactive = get_active_inactive(spine_dFoF[:, i], spine_activity[:, i])
    # d_active, d_inactive = get_active_inactive(dendrite_dFoF[:, i], dendrite_activity[:, i])
    spine_active[:, i] = s_active
    spine_inactive[:, i] = s_inactive
    #dend_active[:, i] = d_active
    #dend_inactive[:, i] = d_inactive

In [6]:
# Select the spine idx to look at
spine_idxs = [39, 48]

In [7]:
# Visualize the activity
%matplotlib qt

fig, ax = plt.subplot_mosaic("""A""",figsize=(7,3))
title = f"{mouse_id}_FOV2_spine_local_coactivity"
fig.suptitle(title)

frames = np.arange(len(spine_active))
for i, spine in enumerate(spine_idxs):
    ax["A"].plot(frames[:], spine_inactive[:, spine] + i, color="blue", alpha=0.3, linewidth=.8)
    ax["A"].plot(frames[:], spine_active[:, spine] + i, color="blue", linewidth=.8)

In [10]:
# Specify the frame range to process the video
frame_range = (40000, 50000)
# Replot
fig, ax = plt.subplot_mosaic("""A""",figsize=(7,3))
title = f"{mouse_id}_FOV2_spine_local_coactivity"
fig.suptitle(title)
frames = np.arange(len(spine_active))
for i, spine in enumerate(spine_idxs):
    ax["A"].plot(frames[frame_range[0]:frame_range[1]], (spine_inactive[frame_range[0]:frame_range[1], spine]) / 2 + i, color="blue", alpha=0.3, linewidth=.8)
    ax["A"].plot(frames[frame_range[0]:frame_range[1]], (spine_active[frame_range[0]:frame_range[1], spine]) / 2 + i, color="blue", linewidth=.8)

In [8]:
# Get active indexes
from Lab_Analyses.Utilities.activity_timestamps import get_activity_timestamps

In [ ]:
spine_idxs = [39,48]
a_indexs = get_activity_timestamps(spine_activity[frame_range[0]:frame_range[1], 48])
a_indexes = [(x[0]-6, x[1]-6) for x in a_indexs]
print(a_indexes)

In [ ]:
# Set up video paths
image_dir = r"Z:\People\Jake\Imaging_Data\JW306\240326\FOV2\GreenCh"
out_name = f"JW306_FOV2_frame_{frame_range[0]}_{frame_range[1]}"
save_dir = r"C:\Users\Jake\Desktop\Figures\Examples\Example_videos"
smooth_window = 30 # In frames

In [ ]:
# Smooth the images to match processed dFoF and trim to only frames needed
frame_range = (frame_range[0] + 229, frame_range[1] + 229)
smooth_trim_video(
    image_dir=image_dir,
    out_name=out_name,
    save_dir=save_dir,
    frame_range=frame_range,
    smooth_window=smooth_window,
)

In [ ]:
# Specify image path to load in after further processing in imageJ
load_image = r"C:\Users\Jake\Desktop\Figures\Examples\Example_videos\JW306_FOV2_frame_40000_50000_cropped_outline_rgb_timer_spines.tif"
image = sio.imread(load_image, plugin="tifffile")

In [ ]:
# Animate plots

## Function to update line plots
def update_lines(num, active, active_line, inactive, inactive_line, images, spine_image):
    active_line.set_data(x[:num], active[:num])
    inactive_line.set_data(x[:num], inactive[:num])
    spine_image.set_array(images[num, :, :, :])
    
    return active_line, inactive_line, spine_image,


plot_inactivity = spine_inactive[frame_range[0]:frame_range[1], spine_idx] / 2
plot_activity = spine_active[frame_range[0]:frame_range[1], spine_idx] / 2
x = np.arange(len(plot_activity))

def init():
    spine_image.set_data(image[0, :, :, :])
    active_line.set_data([], [])
    inactive_line.set_data([], [])
    
    return active_line, inactive_line, spine_image,


fig, axes = plt.subplot_mosaic(
    """AB""",
    figsize=(10,3),
    width_ratios=[1,2.5]
)

# Initialize line plots
active_line, = axes["B"].plot([], [], color="blue", alpha=1, linewidth=.8, label="active")
inactive_line, = axes["B"].plot([], [], color="blue", alpha=0.3, linewidth=.8, label="inactive")

# Initizalie the image
spine_image = axes["A"].imshow(image[0, :, :, :])




axes["B"].legend([active_line, inactive_line], [active_line.get_label(), inactive_line.get_label()], loc=0)
axes["B"].set_xlabel("Frames")
axes["B"].set_ylabel("dF/F")
axes["B"].set_xlim(0, x[-1])
axes["B"].set_ylim(-0.1, 3.1)
axes["A"].axis("off")

fig.tight_layout()

# Setup the Writer for saving
Writer = animation.FFMpegWriter(fps=60)

line_animation = animation.FuncAnimation(
    fig, update_lines, init_func=init, frames=len(plot_inactivity), 
    fargs=(plot_activity, active_line, plot_inactivity, inactive_line, image, spine_image,), 
    interval=5, blit=True, repeat=True
)

animation_fname = r"C:\Users\Jake\Desktop\Figures\Examples\Example_videos\JW306_FOV2_Spine_40.avi"

#line_animation.save(animation_fname, writer=Writer)

In [ ]:
plt.imshow(image[0, :, :, :])

In [ ]:
image.shape

In [ ]:
plt.imshow(np.ones((200,211,3)))

In [ ]:
a = np.array([2,1,1,2,4,5,1,7,])
np.unique(a)